In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w7-p2/sample_submission.csv
/kaggle/input/2021-ai-w7-p2/x_test.csv
/kaggle/input/2021-ai-w7-p2/y_train.csv
/kaggle/input/2021-ai-w7-p2/x_train.csv


In [2]:
import torch
import random

In [3]:
device='cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
random.seed(777)
if device == 'cuda': torch.cuda.manual_seed_all(777)

In [4]:
LR=0.001
epochs=35
BS=126
drop=0.3

In [5]:
xtrain=pd.read_csv('/kaggle/input/2021-ai-w7-p2/x_train.csv',engine='python',encoding='CP949')
ytrain=pd.read_csv('/kaggle/input/2021-ai-w7-p2/y_train.csv',engine='python',encoding='CP949')
test=pd.read_csv('/kaggle/input/2021-ai-w7-p2/x_test.csv',engine='python',encoding='CP949')
submit=pd.read_csv('/kaggle/input/2021-ai-w7-p2/sample_submission.csv',engine='python',encoding='CP949')
print(xtrain.shape)
print(submit.head())

(21486, 14)
   id  predict
0   0        0
1   1        0
2   2        0
3   3        0
4   4        0


In [6]:
xtrain

,시도명,읍면동명,거주인구,근무인구,방문인구,총 유동인구,평균 속도,평균 소요 시간,평균 기온,일강수량,평균 풍속,월,일,년
0,0,26,76018.965,5009.811,36887.341,117916.117,40.467,21.667,17.750,0.000,2.725,9,25,2019
1,1,24,1232416.968,76043.955,451558.268,1760019.191,38.126,31.632,13.300,14.500,4.500,3,4,2018
2,1,23,134260.946,8412.761,78177.981,220851.688,51.107,45.643,5.250,0.500,4.950,4,7,2018
3,1,25,289327.429,39102.424,287029.303,615459.156,33.067,48.000,26.300,0.500,1.500,8,14,2019
4,1,7,1116617.660,96560.651,524905.301,1738083.612,40.392,26.961,8.400,44.000,4.300,12,1,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21481,0,6,91977.598,14802.819,79885.446,186665.863,41.676,21.676,6.400,0.000,1.600,3,5,2020
21482,0,9,63673.610,3725.663,65813.385,133212.658,48.737,26.105,18.300,7.000,3.700,5,26,2018
21483,1,3,406035.642,46335.550,258079.137,710450.329,49.314,50.943,-6.133,0.833,6.067,1,25,2018
21484,1,25,315737.894,25379.224,283079.548,624196.666,35.312,43.688,8.600,0.000,3.100,3,17,2019


In [7]:
# print(ytrain.values)
# print(xtrain.max())
# print(xtrain.min())
# # print(test.max())
# print(xtrain.min())
# print(test.min())

In [8]:
# from sklearn.preprocessing import RobustScaler
# scaler=RobustScaler()
# x=scaler.fit_transform(xtrain)
# xtest=scaler.transform(test)

In [9]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x=scaler.fit_transform(xtrain)
xtest=scaler.transform(test)

In [10]:
x=torch.FloatTensor(x).to(device)
y=torch.FloatTensor(ytrain.values).to(device)
xtest=torch.FloatTensor(xtest).to(device)

In [11]:
dataset=torch.utils.data.TensorDataset(x,y)
dataloader=torch.utils.data.DataLoader(dataset,batch_size=BS,shuffle=True)

In [12]:
linear1=torch.nn.Linear(14,512,bias=True)
linear2=torch.nn.Linear(512,512,bias=True)
linear3=torch.nn.Linear(512,512,bias=True)
linear4=torch.nn.Linear(512,512,bias=True)
linear5=torch.nn.Linear(512,1,bias=True)
relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(p=drop)

In [13]:
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[-3.6713e-02,  6.7380e-03, -5.2585e-03, -1.7382e-02,  2.8180e-02,
          1.0887e-01, -2.4962e-02,  1.1782e-01, -1.6760e-02,  4.2989e-02,
          3.8647e-02,  1.4685e-02,  1.0353e-01, -4.7054e-02,  6.9840e-02,
         -4.7351e-02, -6.5643e-02,  4.2353e-02, -5.7360e-02,  5.0114e-02,
         -2.7609e-02,  1.1161e-01, -1.4944e-01, -4.9749e-02, -6.7113e-03,
          2.0931e-02, -6.8532e-02, -9.9546e-02,  6.7625e-02, -1.8560e-02,
          1.2371e-02,  3.5323e-02,  7.0682e-02,  8.7220e-02, -7.0504e-02,
         -1.2751e-01,  1.7246e-02, -7.1924e-03, -9.9854e-02,  1.0234e-01,
          1.1197e-01,  1.7737e-02, -2.4713e-02, -4.5214e-02, -6.6245e-02,
         -3.3088e-02,  2.4879e-02,  8.8858e-02, -8.6008e-02,  5.3048e-02,
          5.5597e-02, -2.9830e-02, -9.3625e-02, -3.2320e-02, -1.7316e-02,
         -3.4877e-02,  4.7657e-02, -1.6106e-02,  3.2903e-03,  3.9246e-02,
          8.1403e-02, -2.0843e-02, -7.2599e-02,  3.8420e-02,  6.0377e-02,
          1.9045

In [14]:
model=torch.nn.Sequential(
    linear1,relu,dropout,
    linear2,relu,dropout,
    linear3,relu,dropout,
    linear4,relu,dropout,
    linear5).to(device)

In [15]:
loss=torch.nn.MSELoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=LR)

In [16]:
le=len(dataloader)
model.train()

for epoch in range(epochs):
    mcost=0
    for X, Y in dataloader:
        optimizer.zero_grad()
        cost=loss(model(X),Y)
        cost.backward()
        optimizer.step()
        mcost+=cost
    print(epoch,'cost {:.9f}'.format(mcost))

0 cost 30580642.000000000
1 cost 11580235.000000000
2 cost 7439403.500000000
3 cost 5515166.500000000
4 cost 4909148.000000000
5 cost 4356822.500000000
6 cost 3892786.500000000
7 cost 3610476.750000000
8 cost 3502988.250000000
9 cost 3219522.250000000
10 cost 2970455.750000000
11 cost 3069013.250000000
12 cost 2849605.250000000
13 cost 2790963.500000000
14 cost 2692620.500000000
15 cost 2614501.500000000
16 cost 2492942.750000000
17 cost 2505599.500000000
18 cost 2433409.250000000
19 cost 2539166.000000000
20 cost 2349955.000000000
21 cost 2347884.250000000
22 cost 2301138.250000000
23 cost 2327777.250000000
24 cost 2176749.250000000
25 cost 2205828.500000000
26 cost 2116301.500000000
27 cost 2108925.500000000
28 cost 2074116.500000000
29 cost 2143676.000000000
30 cost 2053916.250000000
31 cost 1958842.375000000
32 cost 2018194.125000000
33 cost 1989759.250000000
34 cost 2042965.250000000


In [17]:
with torch.no_grad():
    model.eval()
    prediction=model(x)
    print(prediction)
    print(loss(model(x),y))

tensor([[   2.8268],
        [1490.1061],
        [ 622.0156],
        ...,
        [  -7.3519],
        [ 290.1475],
        [ 231.6946]], device='cuda:0')
tensor(7977.3706, device='cuda:0')


In [18]:
with torch.no_grad():
    model.eval()
    prediction=model(xtest)

In [19]:
submit['predict']=prediction.cpu().numpy()
print(submit.head())

   id      predict
0   0    22.623474
1   1    -0.940294
2   2     1.819673
3   3  1011.861755
4   4  1779.538086


In [20]:
submit.to_csv('submission.csv',index=False,mode='w')